# ⚛️ React Components & Modern Patterns

## Advanced React Skills for Technical Interviews

This notebook demonstrates React expertise with **instant execution** and **traditional autocomplete**.

### Skills Demonstrated:
- ✅ Modern Hooks (useState, useEffect, useCallback, useMemo)
- ✅ Custom Hooks
- ✅ Component Composition
- ✅ Performance Optimization
- ✅ TypeScript Integration
- ✅ State Management Patterns
- ✅ Error Boundaries

## 1. Modern React Hooks Patterns

In [ ]:
// Note: This demonstrates React patterns - in actual interview,
// you'd use a React playground or CodeSandbox for rendering

interface User {
  id: number;
  name: string;
  email: string;
}

interface TodoItem {
  id: number;
  text: string;
  completed: boolean;
  priority: 'low' | 'medium' | 'high';
  createdAt: Date;
}

// Custom hook for data fetching with loading/error states
const useFetch = <T>(url: string, deps: any[] = []) => {
  const [data, setData] = React.useState<T | null>(null);
  const [loading, setLoading] = React.useState(true);
  const [error, setError] = React.useState<string | null>(null);

  React.useEffect(() => {
    const fetchData = async () => {
      try {
        setLoading(true);
        setError(null);
        // Simulated API call
        await new Promise(resolve => setTimeout(resolve, 1000));
        const response = await fetch(url);
        if (!response.ok) throw new Error('Failed to fetch');
        const result = await response.json();
        setData(result);
      } catch (err) {
        setError(err instanceof Error ? err.message : 'Unknown error');
      } finally {
        setLoading(false);
      }
    };

    fetchData();
  }, deps);

  return { data, loading, error, refetch: () => setData(null) };
};

console.log('✅ Custom useFetch hook defined');
console.log('Features: Loading states, error handling, dependencies');

In [ ]:
// Custom hook for local storage with sync
const useLocalStorage = <T>(key: string, initialValue: T) => {
  const [storedValue, setStoredValue] = React.useState<T>(() => {
    try {
      if (typeof window !== 'undefined') {
        const item = window.localStorage.getItem(key);
        return item ? JSON.parse(item) : initialValue;
      }
      return initialValue;
    } catch (error) {
      console.error(`Error reading localStorage key "${key}":`, error);
      return initialValue;
    }
  });

  const setValue = React.useCallback((value: T | ((val: T) => T)) => {
    try {
      const valueToStore = value instanceof Function ? value(storedValue) : value;
      setStoredValue(valueToStore);
      if (typeof window !== 'undefined') {
        window.localStorage.setItem(key, JSON.stringify(valueToStore));
      }
    } catch (error) {
      console.error(`Error setting localStorage key "${key}":`, error);
    }
  }, [key, storedValue]);

  return [storedValue, setValue] as const;
};

// Custom hook for debounced values
const useDebounce = <T>(value: T, delay: number): T => {
  const [debouncedValue, setDebouncedValue] = React.useState(value);

  React.useEffect(() => {
    const handler = setTimeout(() => {
      setDebouncedValue(value);
    }, delay);

    return () => {
      clearTimeout(handler);
    };
  }, [value, delay]);

  return debouncedValue;
};

console.log('✅ Additional custom hooks defined:');
console.log('- useLocalStorage: Persistent state with localStorage sync');
console.log('- useDebounce: Debounced value updates for performance');

## 2. Performance Optimization Techniques

In [ ]:
// Memoization with React.memo and custom comparison
interface ListItemProps {
  item: TodoItem;
  onToggle: (id: number) => void;
  onDelete: (id: number) => void;
}

const ExpensiveListItem = React.memo<ListItemProps>(
  ({ item, onToggle, onDelete }) => {
    // Simulate expensive computation
    const expensiveValue = React.useMemo(() => {
      console.log(`Computing expensive value for item ${item.id}`);
      return item.text.split('').reverse().join('') + item.priority;
    }, [item.text, item.priority]);

    const handleToggle = React.useCallback(() => {
      onToggle(item.id);
    }, [item.id, onToggle]);

    const handleDelete = React.useCallback(() => {
      onDelete(item.id);
    }, [item.id, onDelete]);

    // Virtual DOM representation (normally JSX)
    return {
      type: 'div',
      props: {
        className: `todo-item ${item.completed ? 'completed' : ''}`,
        children: [
          item.text,
          ` (processed: ${expensiveValue})`,
          ' [Toggle] [Delete]'
        ]
      }
    };
  },
  // Custom comparison function
  (prevProps, nextProps) => {
    return (
      prevProps.item.id === nextProps.item.id &&
      prevProps.item.text === nextProps.item.text &&
      prevProps.item.completed === nextProps.item.completed &&
      prevProps.item.priority === nextProps.item.priority
    );
  }
);

console.log('✅ Performance optimization techniques:');
console.log('- React.memo with custom comparison');
console.log('- useMemo for expensive computations');
console.log('- useCallback for stable function references');

## 3. State Management with useReducer

In [ ]:
// Reducer pattern for complex state management
interface AppState {
  todos: TodoItem[];
  filter: 'all' | 'active' | 'completed';
  loading: boolean;
  error: string | null;
}

type AppAction =
  | { type: 'ADD_TODO'; payload: { text: string; priority: TodoItem['priority'] } }
  | { type: 'TOGGLE_TODO'; payload: { id: number } }
  | { type: 'DELETE_TODO'; payload: { id: number } }
  | { type: 'SET_FILTER'; payload: { filter: AppState['filter'] } }
  | { type: 'SET_LOADING'; payload: { loading: boolean } }
  | { type: 'SET_ERROR'; payload: { error: string | null } };

const appReducer = (state: AppState, action: AppAction): AppState => {
  switch (action.type) {
    case 'ADD_TODO':
      const newTodo: TodoItem = {
        id: Math.max(0, ...state.todos.map(t => t.id)) + 1,
        text: action.payload.text,
        completed: false,
        priority: action.payload.priority,
        createdAt: new Date()
      };
      return {
        ...state,
        todos: [...state.todos, newTodo]
      };
      
    case 'TOGGLE_TODO':
      return {
        ...state,
        todos: state.todos.map(todo =>
          todo.id === action.payload.id
            ? { ...todo, completed: !todo.completed }
            : todo
        )
      };
      
    case 'DELETE_TODO':
      return {
        ...state,
        todos: state.todos.filter(todo => todo.id !== action.payload.id)
      };
      
    case 'SET_FILTER':
      return {
        ...state,
        filter: action.payload.filter
      };
      
    default:
      return state;
  }
};

// Demo usage
const initialState: AppState = {
  todos: [],
  filter: 'all',
  loading: false,
  error: null
};

console.log('✅ State management implemented:');
console.log('- useReducer for complex state logic');
console.log('- Type-safe action definitions');
console.log('- Immutable state updates');
console.log('- Centralized state management');

// Test the reducer
const testState = appReducer(initialState, {
  type: 'ADD_TODO',
  payload: { text: 'Learn React patterns', priority: 'high' }
});

console.log('Demo state after adding todo:', testState);

## 4. Error Boundaries & Error Handling

In [ ]:
// Error Boundary class component (conceptual - would need React DOM)
interface ErrorBoundaryState {
  hasError: boolean;
  error: Error | null;
  errorInfo: any;
}

// Conceptual Error Boundary (would extend React.Component in real app)
class ErrorBoundary {
  state: ErrorBoundaryState = {
    hasError: false,
    error: null,
    errorInfo: null
  };

  static getDerivedStateFromError(error: Error): Partial<ErrorBoundaryState> {
    return {
      hasError: true,
      error
    };
  }

  componentDidCatch(error: Error, errorInfo: any) {
    // Log error to monitoring service
    console.error('Error Boundary caught an error:', {
      error: error.message,
      stack: error.stack,
      componentStack: errorInfo.componentStack
    });
    
    this.state = {
      hasError: true,
      error,
      errorInfo
    };
  }

  render() {
    if (this.state.hasError) {
      return {
        type: 'div',
        props: {
          className: 'error-boundary',
          children: [
            'Something went wrong.',
            `Error: ${this.state.error?.message}`,
            'Please refresh the page.'
          ]
        }
      };
    }
    
    return null; // Would return this.props.children in real component
  }
}

// Error handling hook
const useErrorHandler = () => {
  const [error, setError] = React.useState<Error | null>(null);
  
  const handleError = React.useCallback((error: Error) => {
    console.error('Caught error:', error);
    setError(error);
    
    // Could send to error reporting service here
  }, []);
  
  const clearError = React.useCallback(() => {
    setError(null);
  }, []);
  
  return { error, handleError, clearError };
};

console.log('✅ Error handling implemented:');
console.log('- Error Boundary for component errors');
console.log('- Custom error handling hook');
console.log('- Error logging and reporting');
console.log('- Graceful error recovery');

// Demo error handling
const errorHandler = {
  error: null,
  handleError: (err: Error) => {
    console.log('Handling error:', err.message);
  },
  clearError: () => {
    console.log('Error cleared');
  }
};

try {
  throw new Error('Demo error for testing');
} catch (err) {
  errorHandler.handleError(err as Error);
}

console.log('Error handling demo completed');

## 📊 React Skills Summary

### ✅ Advanced React Patterns Demonstrated:
- **Custom Hooks**: useFetch, useLocalStorage, useDebounce
- **Performance Optimization**: React.memo, useMemo, useCallback
- **State Management**: useReducer with type-safe actions
- **Error Handling**: Error boundaries and error handling hooks
- **TypeScript Integration**: Strong typing throughout

### 💪 Technical Competencies:
- Memory-efficient programming with memoization
- Complex state management patterns
- Error boundary implementation
- Custom hook development
- Performance optimization techniques

### 🎯 Interview Readiness:
All patterns demonstrate production-ready React skills with traditional autocomplete support - no AI assistance required!

### 🚀 Key Takeaways:
1. **Hooks Mastery**: Advanced usage of built-in and custom hooks
2. **Performance Focus**: Optimization techniques for large applications
3. **Error Resilience**: Comprehensive error handling strategies
4. **Type Safety**: Full TypeScript integration
5. **Best Practices**: Production-ready code patterns